In [ ]:
import numpy as np
import pandas as pd
import datetime
import pytz

# np.zeros((5,4))
# np.random.randint(1,7,(5,4))
df = pd.DataFrame(np.random.randint(0,100,size=(3, 4)), columns=list('ABCD'))
df.iloc[0] = [np.nan, np.nan, np.nan, 'nan']
print(df.isnull().sum().sum())
display(df)

df2 = df[['A','B']]
df2['E'] = df2['A']+1
display(df2)
display(df)


In [ ]:
pd.concat([df, pd.DataFrame(np.array([1,2,3]))], axis=1, ignore_index=True)

In [ ]:
from collections import Counter

c = Counter([1,2,3])
c[4]

In [ ]:
import sys
-sys.maxsize

#nan based assert
a, b = np.nan, np.nan
a, b = 1,1
# a, b = 1,np.nan
assert ((np.isnan(a) and np.isnan(b)) or (a == b))


In [ ]:
#Datetime functions

# import pandas as pd
# import datetime
# from dateutil.relativedelta import relativedelta

# class Const:
#     cur_date = datetime.datetime.today()
#     prior_date = (datetime.datetime.today() + relativedelta(months=-1))
#     date_ranges = pd.date_range(start=prior_date.strftime("%m/%d/%Y"), end=cur_date.strftime("%m/%d/%Y"))

import io
data = {'date':['03/28/2019  10:08:44 PM','4/26/2019  12:43:07 AM','4/35/2019  12:43:07 AM','',0,'0']}
df = pd.DataFrame(data)
# df = pd.read_csv(io.StringIO(data))
# encoding='latin1')

df['f_date'] = pd.to_datetime(df['date'], errors='coerce')

max_date = max(df['f_date'])
date_val = max_date - pd.to_timedelta(50, 'D') 
df['flag'] = (df['f_date'] >= date_val)

print(df)
print(max_date, date_val)

In [ ]:
#utc time now

print(pd.to_datetime('now').strftime('UTC-%Y-%m-%d %H:%M:%S'))
print(datetime.datetime.utcnow().strftime('UTC-%Y-%m-%d %H:%M:%S'))
print(datetime.datetime.now(pytz.utc).strftime("%Y-%m-%d %H:%M:%S"))

## Multiprocessing

In [ ]:
# https://www.journaldev.com/15631/python-multiprocessing-example

## Bash

In [ ]:
# export PS1='\!:\[\033[0;32m\]$( basename `git rev-parse --show-toplevel 2> /dev/null` 2> /dev/null | sed "s/^/[/")$( git branch 2> /dev/null | cut -f2 -d\* -s | sed "s/^ /:/" | sed "s/$/]/" )\[\033[0m\]$( pwd | sed "s|`git rev-parse --show-toplevel 2> /dev/null`||" 2> /dev/null || dirs +0) \$ '


## AWS Cloud

In [ ]:
#Read many part files from AWS s3

import boto3, io, s3fs
from multiprocessing import Pool

## Connections

# conn = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='r1')

# cursor = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='r1').cursor()

## --------------------------------------------------
## Pandas batch style based data fetch (Error: Query times out for large datasets)

# def read_data_from_athena(db_name, table_name):
#     query = "SELECT * FROM {}.{}".format(db_name, table_name) 
#     cur_df = pd.read_sql(query, conn)
#     return cur_df

# raw_training_df = read_data_from_athena(db_name, table_name)

## --------------------------------------------------
## Cursor based data fetch (Error: No result set for large datasets)

# def get_cursor_iterator(db_name, table_name):
#     query = "SELECT * FROM {}.{} limit 10".format(db_name, table_name)
#     print(query)
#     for row in cursor:
#         print(row)
#         break

# get_cursor_iterator(db_name, table_name)

## --------------------------------------------------
def serial_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    sub_df = []
    for obj in prefix_objs:
        file_key = obj.key
        print(file_key)
        cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
        cur_df = cur_df[valid_cols]
        sub_df.append(cur_df)
    return pd.concat(sub_df)

def get_data(bucket_name, file_key, valid_cols):
    print(file_key)
    cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
    cur_df = cur_df[valid_cols]
    return cur_df

class GET_DATA(): #Helper class to invoke get_data() as local lambda's cannot be pickled so cannot be used with pool.map. functools.partial can also be used 
    def __init__(self, bucket_name, valid_cols):
        self.bucket_name = bucket_name
        self.valid_cols = valid_cols
    def __call__(self, file_key):
        return get_data(self.bucket_name, file_key, self.valid_cols)

def parallel_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    file_keys = tuple(map(lambda x : x.key, prefix_objs))
    agents, chunksize = 5, 25
    GD = GET_DATA(bucket_name, valid_cols)
    with Pool(processes=agents) as pool:
        sub_df = pool.map(GD, file_keys, chunksize)
    return pd.concat(sub_df)

#------------------------------------------------------------------
s3 = boto3.client('s3')

def read_df_from_s3(file_path, header_flag=True):
    obj = s3.get_object(Bucket=bucket_name, Key=file_path)
    if header_flag:
        tmp_df = pd.read_csv(BytesIO(obj['Body'].read()))
    else:
        tmp_df = pd.read_csv(BytesIO(obj['Body'].read()), header=None)
    return tmp_df

def write_df_to_s3(cur_df, bucket_name, file_key):
    csv_buffer = StringIO()
    cur_df.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name, file_key).put(Body=csv_buffer.getvalue())


In [ ]:
#AWS cli commands

aws batch submit-job --generate-cli-skeleton
aws lambda update-function-code --function-name "abc" --s3-bucket "def" --s3-key "code/ghi.zip"
aws ec2 describe-vpcs
aws ec2 describe-nat-gateways

aws emr create-cluster \
  --release-label emr-5.25.0 \
  --name 'abc' \
  --applications Name=Hadoop Name=Hive Name=Spark Name=Ganglia \
  --tags 'def=ghi' \
  --ec2-attributes KeyName=k1,SubnetId=s1,EmrManagedMasterSecurityGroup=s2,EmrManagedSlaveSecurityGroup=s3,ServiceAccessSecurityGroup=s4,InstanceProfile=p1 \
  --service-role EMR_DefaultRole --enable-debugging \
  --instance-groups \
    InstanceGroupType=MASTER,InstanceCount=1,InstanceType=m5.2xlarge \
    InstanceGroupType=CORE,InstanceCount=1,InstanceType=r5.12xlarge,BidPrice=b \
  --region r1 \
  --log-uri s2://u1

aws emr add-steps --cluster-id abc --steps file://def.json
        

In [ ]:
#Script ec2 creation

import boto3
import datetime
import os
import time


ec2 = boto3.resource('ec2')
REGION = os.getenv('region', 'r1')
ENV = os.getenv('env', 'defaultenv')
iam_instance_profile = os.getenv('iam_instance_profile', '')


def handler(event, context):
    response = ec2.create_instances(
        BlockDeviceMappings=[
            {
                'DeviceName': '/dev/sda1', #EBS-optimized has become False due to this change
                'Ebs': {
                    'DeleteOnTermination': True,
                    'VolumeSize': 50
                    #'VolumeType': 'gp2'
                }
            },
        ],        
        ImageId='ami-....ubuntu',
        InstanceType='',
        KeyName='',
        MaxCount=1,
        MinCount=1,
        Monitoring={
            'Enabled': True
        },
        SecurityGroupIds=[
            '',
        ],
        SubnetId='',
        IamInstanceProfile={
            'Arn': iam_instance_profile,
        },
        InstanceInitiatedShutdownBehavior='terminate',
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'k1',
                        'Value': v1
                    },
                ]
            },
        ]
    )
    print(response)
    try:
        if len(response) > 0:
            return {'status': 'OK', 'instance_id': response[0].id,
                    'ResponseMetadata': {'response_timestamp': int(time.time()),
                                         'response_datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}
    except:
        print('Error occurred during instance creation...')
    return {'status': 'ERROR', 'error_message': 'Error occurred while creating EC2 instance.',
            'ResponseMetadata': {'response_timestamp': int(time.time()),
                                 'response_datetime': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}


if __name__ == '__main__':
    handler("","")

# Start jupyter lab without authentication    
jupyter lab --ip=* --NotebookApp.token=''    

# Within aws lambda
os.system("unzip -x /tmp/amlbins.zip -d /tmp/amlbins >/dev/null 2>&1") #NOTE: redirecting stdout, stderr to null so as to easily debug lambda logs on cloudwatch


## Monitoring

In [ ]:
##Publish metrics to cloudwatch metrics
# https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/cloudwatch_concepts.html

d = {1:2, 3:4, 5:6}

import boto3
cloudwatch = boto3.client('cloudwatch')

def is_potential_float(v):
    try:
        float(v)
        return True
    except:
        return False
    
#Rather than differentiating dev/prod metrics by dimensions lets push dev/prod metrics to different namespaces
def iterate_dict(d, cur_metric_data):
    for k,v in d.items():
        if isinstance(v, dict):
            iterate_dict(v, cur_metric_data)
        elif is_potential_float(v):
            v = float(v)
            cur_metric_data.append(
                {
                    'MetricName': k,
                    'Unit': 'None',
                    'Value': v
                }
            )

def publish_cw_metricdata_from_dict(d):
    cur_metric_data = []
    cur_namespace = 'a/b'
    iterate_dict(d, cur_metric_data)
    #print(cur_metric_data)
    response = cloudwatch.put_metric_data(
        MetricData = cur_metric_data,
        Namespace = cur_namespace
    )
    return response

response = publish_cw_metricdata_from_dict(d)
response


In [ ]:
##Publish metrics to Prometheus
# https://github.com/prometheus/client_python

##Publish metrics to elastic search
# https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/


## Docker

In [ ]:
#Extract docker tag, digest from AWS ECR

import boto3
import docker
import json

repository_name = 'repoparent/reponame'
session = boto3.Session(profile_name='default')
client = boto3.client('ecr')
# docker_api = docker.APIClient()
print(session)

response = client.batch_get_image(
    repositoryName=repository_name,
    imageIds=[
        {
            'imageTag': ''
        },
    ]
)

# response = client.batch_get_image(
#     repositoryName=repository_name,
#     imageIds=[
#         {
#             'imageDigest': 'sha256:...'
#         },
#     ]
# )

# response = client.list_images(
#     repositoryName=repository_name,
#     maxResults=5,
#     filter={
#         'tagStatus': 'TAGGED'
#     }
# )

# from IPython.display import JSON
# JSON(response, expanded=True)

# import hashlib
# print(hashlib.sha256(b"abc123").hexdigest())

# print(response['images'][0]['imageId']['imageDigest'])
print(json.dumps(response,indent=1))

In [ ]:
#Get 2 tag names for the `recent` env specific docker tag having same image digest

env_name = 'non-prod'
response = client.batch_get_image(
    repositoryName=repository_name,
    imageIds=[
        {
            'imageTag': env_name
        },
    ]
)
assert len(response['images']) == 1, "1 recent docker is expected with tag {} under {} ecr repository".format(env_name, repository_name)
image_digest = response['images'][0]['imageId']['imageDigest']

response = client.batch_get_image(
    repositoryName=repository_name,
    imageIds=[
        {
            'imageDigest': image_digest
        },
    ]
)
assert len(response['images']) == 2, "2 tags are expected for docker with image_digest {} under {} ecr repository".format(image_digest, repository_name)
tag_digest_list = [(response['images'][idx]['imageId']['imageTag'], response['images'][idx]['imageId']['imageDigest']) 
                   for idx in range(2)]
print(tag_digest_list)

## Kafka

## Avro Message

## Gitlab CI/CD

## VirtualEnv